In [1]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from collections import deque
from yolox.tracker.byte_tracker import BYTETracker
from torchvision import transforms, models
from ultralytics import YOLO
from types import SimpleNamespace

# --- تعریف مدل سیامی ---
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet50(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.embedding = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 128),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = self.pool(x)
        x = self.embedding(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2

# --- CONFIG ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIAMESE_MODEL_PATH = r"F:\mtmctwithsiamese\siameselast.pth"
YOLO_MODEL_PATH = "yolov8n.pt"
VIDEO_PATHS = [r"F:\mtmctwithsiamese\22.mp4", r"F:\mtmctwithsiamese\11.mp4"]
IMG_SIZE = (224, 224)
FRAME_BUFFER = 40
CONF_THRESH = 0.15
TRACK_BUFFER = 120
DISTANCE_THRESH = 0.45
INACTIVE_DISTANCE_THRESH = 0.7
MIN_BOX_SIZE = 40
IOU_THRESHOLD = 0.6
REVISIT_INTERVAL = 120
REVISIT_COOLDOWN = 30
DEBUG_SAVE = True
NUM_COMPARISON_IMAGES = 5
MIN_CONFIDENCE = 0.4
DUPLICATE_GID_THRESHOLD = 3  # تعداد فریم‌ها برای تشخیص GID تکراری

# --- پیش‌پردازش تصویر ---
def get_adaptive_transform():
    return transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# --- تنظیمات BYTETracker ---
args = SimpleNamespace(
    track_thresh=0.05,
    track_buffer=TRACK_BUFFER,
    match_thresh=0.8,
    min_box_area=MIN_BOX_SIZE * MIN_BOX_SIZE,
    mot20=False
)

# --- بارگذاری مدل سیامی ---
model = SiameseNetwork().to(DEVICE)
try:
    checkpoint = torch.load(SIAMESE_MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find Siamese model at {SIAMESE_MODEL_PATH}")

# --- بارگذاری مدل YOLO ---
def load_yolo_model(model_path):
    model = YOLO(model_path)
    model.to(DEVICE)
    return model

yolo_model = load_yolo_model(YOLO_MODEL_PATH)

# --- رجیستری جهانی ---
global_id_counter = 0
camera_registry = {0: {}, 1: {}}
inactive_tracks = {0: {}, 1: {}}
used_gids = {0: set(), 1: set()}
gid_history = {}
duplicate_gids = {0: {}, 1: {}}  # برای ردیابی GIDهای تکراری در هر دوربین

# --- توابع کمکی ---
def preprocess_image(img):
    transform = get_adaptive_transform()
    return transform(img)

def compute_siamese_distance(img1, img2, model):
    img1_tensor = preprocess_image(img1).unsqueeze(0).to(DEVICE)
    img2_tensor = preprocess_image(img2).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        emb1, emb2 = model(img1_tensor, img2_tensor)
        distance = torch.nn.functional.pairwise_distance(emb1, emb2).item()
    return distance

def compute_average_distance(new_img, other_frames, model):
    if not other_frames:
        return float('inf')
    valid_frames = [img for img in list(other_frames) if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE]
    if not valid_frames:
        return float('inf')
    distances = []
    for img in valid_frames[-NUM_COMPARISON_IMAGES:]:
        dist = compute_siamese_distance(new_img, img, model)
        distances.append(dist)
    return np.mean(distances) if distances else float('inf')

def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1[:4]
    x1_, y1_, x2_, y2_ = box2[:4]
    inter_x1 = max(x1, x1_)
    inter_y1 = max(y1, y1_)
    inter_x2 = min(x2, x2_)
    inter_y2 = min(y2, y2_)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def get_yolo_detections(frame):
    results = yolo_model.predict(frame, conf=CONF_THRESH, classes=[0], verbose=False)
    detections = []
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        print(f"Number of raw detections: {len(boxes)}")
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confs.tolist(), CONF_THRESH, IOU_THRESHOLD)
        if len(indices) > 0:
            indices = indices.flatten()
            for i in indices:
                box = boxes[i]
                conf = confs[i]
                if (box[2] - box[0]) >= MIN_BOX_SIZE and (box[3] - box[1]) >= MIN_BOX_SIZE and conf >= MIN_CONFIDENCE:
                    detections.append(np.concatenate([box, [conf]]))
    if detections:
        return np.array(detections)
    print("No detections found after NMS")
    return np.empty((0, 5))

def assign_global_id(new_img, cam_id, track_id, frame_count, active_tracks, inactive_tracks):
    global global_id_counter
    assigned_id = None
    min_dist = float('inf')
    best_candidate = None

    print(f"Cam {cam_id}, Track {track_id} - Assigning GID at Frame {frame_count}")

    other_cam_id = 1 if cam_id == 0 else 0
    active_gids_in_cam = {info["global_id"] for tid, info in active_tracks[cam_id].items() if info["global_id"] is not None}

    # ابتدا با افراد غیرفعال مقایسه کن
    for other_tid, other_info in inactive_tracks[other_cam_id].items():
        if "frames" in other_info and other_info["frames"]:
            dist = compute_average_distance(new_img, other_info["frames"], model)
            valid_frame_count = len([img for img in other_info["frames"] if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE])
            print(f"Comparing with Inactive Cam {other_cam_id}, Track {other_tid}, GID {other_info['global_id']}, Avg Distance: {dist:.4f}, Valid Frames: {valid_frame_count}/{len(other_info['frames'])}")
            if dist < INACTIVE_DISTANCE_THRESH and dist < min_dist and other_info["global_id"] not in active_gids_in_cam:
                min_dist = dist
                best_candidate = other_info["global_id"]

    # اگر تطبیقی پیدا نشد، با افراد فعال مقایسه کن
    if best_candidate is None:
        for other_tid, other_info in active_tracks[other_cam_id].items():
            if "frames" in other_info and other_info["frames"]:
                dist = compute_average_distance(new_img, other_info["frames"], model)
                valid_frame_count = len([img for img in other_info["frames"] if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE])
                print(f"Comparing with Cam {other_cam_id}, Track {other_tid}, GID {other_info['global_id']}, Avg Distance: {dist:.4f}, Valid Frames: {valid_frame_count}/{len(other_info['frames'])}")
                if dist < DISTANCE_THRESH and dist < min_dist and other_info["global_id"] not in active_gids_in_cam:
                    min_dist = dist
                    best_candidate = other_info["global_id"]

    if best_candidate is not None:
        assigned_id = best_candidate
        print(f"Matched with existing GID: {assigned_id}")
    else:
        assigned_id = global_id_counter
        global_id_counter += 1
        print(f"New GID assigned: {assigned_id}")

    used_gids[cam_id].add(assigned_id)
    gid_history[assigned_id] = {
        "cam_id": cam_id,
        "track_id": track_id,
        "last_distance": min_dist,
        "frame_count": frame_count,
        "frames_assigned": 1,
        "last_revisit_frame": 0
    }
    print(f"Cam {cam_id} - Used GIDs: {used_gids[cam_id]}")

    if DEBUG_SAVE:
        os.makedirs(f"debug_images/cam_{cam_id}", exist_ok=True)
        cv2.imwrite(f"debug_images/cam_{cam_id}/track_{track_id}_frame_{frame_count}_gid_{assigned_id}_dist_{min_dist:.4f}.jpg", new_img)
        with open("distances.txt", "a") as f:
            f.write(f"Frame {frame_count}, Cam {cam_id}, Track {track_id}, GID {assigned_id}, Min Distance: {min_dist:.4f}, Active GIDs in Cam: {active_gids_in_cam}\n")

    print(f"Cam {cam_id}, Track {track_id} - Assigned GID: {assigned_id}")
    return assigned_id

def revisit_global_ids(cam_id, active_tracks, inactive_tracks, frame_count, track_ids=None):
    global global_id_counter  # اضافه کردن متغیر جهانی
    other_cam_id = 1 if cam_id == 0 else 0
    # اگر track_ids مشخص شده باشد، فقط آن‌ها را بازبینی کن
    tracks_to_revisit = track_ids if track_ids else active_tracks[cam_id].keys()
    
    for tid in tracks_to_revisit:
        track_info = active_tracks[cam_id].get(tid)
        if not track_info or not track_info["frames"] or track_info["global_id"] is None:
            continue
        current_img = track_info["frames"][-1]
        current_gid = track_info["global_id"]
        if current_gid not in gid_history:
            continue

        last_revisit = gid_history[current_gid].get("last_revisit_frame", 0)
        if frame_count - last_revisit < REVISIT_COOLDOWN:
            print(f"Cam {cam_id}, Track {tid} - Skipping revisit due to cooldown")
            continue

        frames_assigned = gid_history[current_gid]["frames_assigned"]
        current_dist = gid_history[current_gid]["last_distance"]

        new_gid = None
        min_dist = float('inf')

        # ابتدا با افراد غیرفعال مقایسه کن
        for other_tid, other_info in inactive_tracks[other_cam_id].items():
            if other_info["frames"] and other_info["global_id"] != current_gid:
                dist = compute_average_distance(current_img, other_info["frames"], model)
                valid_frame_count = len([img for img in other_info["frames"] if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE])
                print(f"Revisit: Cam {cam_id}, Track {tid}, Comparing with Inactive Cam {other_cam_id}, Track {other_tid}, GID {other_info['global_id']}, Distance: {dist:.4f}, Valid Frames: {valid_frame_count}")
                if dist < INACTIVE_DISTANCE_THRESH and dist < min_dist:
                    min_dist = dist
                    new_gid = other_info["global_id"]

        # اگر تطبیقی پیدا نشد، با افراد فعال مقایسه کن
        if new_gid is None:
            for other_tid, other_info in active_tracks[other_cam_id].items():
                if other_info["frames"] and other_info["global_id"] != current_gid:
                    dist = compute_average_distance(current_img, other_info["frames"], model)
                    valid_frame_count = len([img for img in other_info["frames"] if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE])
                    print(f"Revisit: Cam {cam_id}, Track {tid}, Comparing with Cam {other_cam_id}, Track {other_tid}, GID {other_info['global_id']}, Distance: {dist:.4f}, Valid Frames: {valid_frame_count}")
                    if dist < DISTANCE_THRESH and dist < min_dist:
                        min_dist = dist
                        new_gid = other_info["global_id"]

        # اگر GID جدیدی پیدا شد و با GID فعلی متفاوت بود
        if new_gid is not None and new_gid != current_gid:
            if min_dist < 0.3 and (frames_assigned < 50 or min_dist < current_dist * 0.5):
                print(f"Reassigning GID {new_gid} to Track {tid} from GID {current_gid}")
                if current_gid in used_gids[cam_id]:
                    used_gids[cam_id].remove(current_gid)
                track_info["global_id"] = new_gid
                used_gids[cam_id].add(new_gid)
                gid_history[new_gid] = {
                    "cam_id": cam_id,
                    "track_id": tid,
                    "last_distance": min_dist,
                    "frame_count": frame_count,
                    "frames_assigned": gid_history.get(new_gid, {}).get("frames_assigned", 0) + 1,
                    "last_revisit_frame": frame_count
                }
                gid_history[current_gid]["frames_assigned"] = 0
                if DEBUG_SAVE:
                    with open("revisit_log.txt", "a") as f:
                        f.write(
                            f"Frame {frame_count}, Cam {cam_id}, Track {tid}: Reassigned GID {new_gid}, "
                            f"Distance: {min_dist:.4f}, Frames Assigned: {frames_assigned}\n"
                        )
        else:
            # اگر GID جدیدی پیدا نشد، یک GID جدید تخصیص بده
            if min_dist > INACTIVE_DISTANCE_THRESH:
                new_gid = global_id_counter
                global_id_counter += 1
                print(f"Assigning new GID {new_gid} to Track {tid} due to high distance")
                if current_gid in used_gids[cam_id]:
                    used_gids[cam_id].remove(current_gid)
                track_info["global_id"] = new_gid
                used_gids[cam_id].add(new_gid)
                gid_history[new_gid] = {
                    "cam_id": cam_id,
                    "track_id": tid,
                    "last_distance": min_dist,
                    "frame_count": frame_count,
                    "frames_assigned": 1,
                    "last_revisit_frame": frame_count
                }
                gid_history[current_gid]["frames_assigned"] = 0
                if DEBUG_SAVE:
                    with open("revisit_log.txt", "a") as f:
                        f.write(
                            f"Frame {frame_count}, Cam {cam_id}, Track {tid}: Assigned new GID {new_gid}, "
                            f"Distance: {min_dist:.4f}, Frames Assigned: {frames_assigned}\n"
                        )

def check_duplicate_gids(cam_id, active_tracks, frame_count, inactive_tracks):
    """بررسی GIDهای تکراری در یک دوربین و اجبار به بازبینی"""
    gid_to_tracks = {}
    for tid, track_info in active_tracks[cam_id].items():
        gid = track_info.get("global_id")
        if gid is not None:
            if gid not in gid_to_tracks:
                gid_to_tracks[gid] = []
            gid_to_tracks[gid].append(tid)

    # بررسی GIDهایی که بیش از یک نفر دارند
    for gid, tracks in gid_to_tracks.items():
        if len(tracks) > 1:
            print(f"Cam {cam_id}, Frame {frame_count} - Duplicate GID {gid} assigned to tracks: {tracks}")
            if gid not in duplicate_gids[cam_id]:
                duplicate_gids[cam_id][gid] = {"frame_count": frame_count, "count": 1}
            else:
                duplicate_gids[cam_id][gid]["count"] += 1
                if duplicate_gids[cam_id][gid]["count"] >= DUPLICATE_GID_THRESHOLD:
                    print(f"Cam {cam_id}, Frame {frame_count} - Triggering revisit for duplicate GID {gid} on tracks {tracks}")
                    # بازبینی برای همه trackهای دارای این GID
                    revisit_global_ids(cam_id, active_tracks, inactive_tracks, frame_count, track_ids=tracks)
                    # ریست کردن شمارشگر
                    duplicate_gids[cam_id][gid]["count"] = 0
                    if DEBUG_SAVE:
                        with open("duplicate_gids_log.txt", "a") as f:
                            f.write(f"Frame {frame_count}, Cam {cam_id}, GID {gid}, Tracks {tracks}: Triggered revisit\n")
        else:
            # اگر دیگر تکراری نیست، شمارشگر را ریست کن
            if gid in duplicate_gids[cam_id]:
                del duplicate_gids[cam_id][gid]

def moving_average_id_assignments(cam_id, assigned_id, track_id, new_img, frame_count):
    """اعمال فیلتر برای جلوگیری از تغییرات ناگهانی در تخصیص GID"""
    global gid_history
    if assigned_id is None or assigned_id not in gid_history:
        return assigned_id

    current_info = gid_history[assigned_id]
    prev_track_id = current_info["track_id"]
    prev_cam_id = current_info["cam_id"]
    
    if prev_cam_id != cam_id or prev_track_id != track_id:
        prev_frames = None
        if prev_cam_id in camera_registry and prev_track_id in camera_registry[prev_cam_id]:
            prev_frames = camera_registry[prev_cam_id][prev_track_id]["frames"]
        elif prev_cam_id in inactive_tracks and prev_track_id in inactive_tracks[prev_cam_id]:
            prev_frames = inactive_tracks[prev_cam_id][prev_track_id]["frames"]
            
        if prev_frames:
            prev_dist = compute_average_distance(new_img, prev_frames, model)
            valid_frame_count = len([img for img in prev_frames if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE])
            print(f"Cam {cam_id}, Track {track_id} - Checking GID {assigned_id} with prev track {prev_track_id}, Distance: {prev_dist:.4f}, Valid Frames: {valid_frame_count}")
            if prev_dist >= INACTIVE_DISTANCE_THRESH:
                global global_id_counter
                new_assigned_id = global_id_counter
                global_id_counter += 1
                print(f"Cam {cam_id}, Track {track_id} - Reassigning new GID {new_assigned_id} due to mismatch with prev track {prev_track_id}, Distance: {prev_dist:.4f}")
                if assigned_id in used_gids[cam_id]:
                    used_gids[cam_id].remove(assigned_id)
                used_gids[cam_id].add(new_assigned_id)
                gid_history[new_assigned_id] = {
                    "cam_id": cam_id,
                    "track_id": track_id,
                    "last_distance": prev_dist,
                    "frame_count": frame_count,
                    "frames_assigned": 1,
                    "last_revisit_frame": 0
                }
                return new_assigned_id
    return assigned_id

def process_frame(frame, tracker, cam_id, frame_count, active_tracks, inactive_tracks):
    detections = get_yolo_detections(frame)
    print(f"Cam {cam_id} - YOLO detections shape: {detections.shape}")

    img_info = [frame.shape[0], frame.shape[1]]
    if detections.shape[0] > 0:
        online_targets = tracker.update(detections, img_info, frame.shape[:2])
    else:
        online_targets = []
    
    print(f"Cam {cam_id} - Number of tracked targets: {len(online_targets)}")

    current_track_ids = set()
    for t in online_targets:
        tid = t.track_id
        current_track_ids.add(tid)
        if tid not in camera_registry[cam_id]:
            camera_registry[cam_id][tid] = {
                "frames": deque(maxlen=FRAME_BUFFER),
                "global_id": None,
                "last_frame": frame_count,
                "confidence": t.score if hasattr(t, 'score') else 0.0
            }
        
        tlbr = t.tlbr.astype(int)
        person_img = frame[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]]
        if person_img.size == 0 or person_img.shape[0] < MIN_BOX_SIZE or person_img.shape[1] < MIN_BOX_SIZE or t.score < MIN_CONFIDENCE:
            print(f"Cam {cam_id} - Invalid person image or low confidence for Track {tid}, Confidence: {t.score:.4f}")
            continue
        
        if DEBUG_SAVE:
            os.makedirs(f"debug_images/cam_{cam_id}/track_{tid}", exist_ok=True)
            cv2.imwrite(f"debug_images/cam_{cam_id}/track_{tid}/frame_{frame_count}.jpg", person_img)
        
        camera_registry[cam_id][tid]["frames"].append(person_img)
        camera_registry[cam_id][tid]["last_frame"] = frame_count
        camera_registry[cam_id][tid]["confidence"] = t.score if hasattr(t, 'score') else 0.0
        print(f"Cam {cam_id}, Track {tid} - Frame buffer size: {len(camera_registry[cam_id][tid]['frames'])}")

        track_info = camera_registry[cam_id][tid]
        valid_imgs = [img for img in track_info["frames"] if img.shape[0] >= MIN_BOX_SIZE and img.shape[1] >= MIN_BOX_SIZE]
        if len(valid_imgs) == 0:
            print(f"Cam {cam_id}, Track {tid} - No valid frames available")
            continue
        latest_img = valid_imgs[-1]

        if len(track_info["frames"]) >= FRAME_BUFFER and track_info["global_id"] is None:
            if len(valid_imgs) < FRAME_BUFFER * 0.7:
                print(f"Cam {cam_id}, Track {tid} - Not enough valid frames: {len(valid_imgs)}/{FRAME_BUFFER}")
                continue
            track_info["global_id"] = assign_global_id(latest_img, cam_id, tid, frame_count, active_tracks, inactive_tracks)
            track_info["global_id"] = moving_average_id_assignments(cam_id, track_info["global_id"], tid, latest_img, frame_count)
        elif track_info["global_id"] is not None:
            gid = track_info["global_id"]
            if gid in gid_history:
                gid_history[gid]["frames_assigned"] += 1
            track_info["global_id"] = moving_average_id_assignments(cam_id, track_info["global_id"], tid, latest_img, frame_count)

        global_id = track_info["global_id"]
        label = f"GID:{global_id}" if global_id is not None else f"TID:{tid}"
        cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), (0, 255, 0), 2)
        cv2.putText(frame, label, (tlbr[0], tlbr[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f"Cam {cam_id} - Drew bounding box for Track {tid}, Label: {label}")
    
    active_tracks[cam_id] = {tid: info for tid, info in camera_registry[cam_id].items() if info["last_frame"] == frame_count}
    
    for tid in list(camera_registry[cam_id].keys()):
        if tid not in current_track_ids and tid not in inactive_tracks[cam_id]:
            if camera_registry[cam_id][tid]["frames"]:
                inactive_tracks[cam_id][tid] = camera_registry[cam_id][tid]
                inactive_tracks[cam_id][tid]["inactive_since"] = frame_count
                print(f"Cam {cam_id}, Track {tid} - Moved to inactive_tracks with GID {inactive_tracks[cam_id][tid]['global_id']}")

    # بررسی GIDهای تکراری
    check_duplicate_gids(cam_id, active_tracks, frame_count, inactive_tracks)

    if frame_count % REVISIT_INTERVAL == 0:
        print(f"Cam {cam_id} - Performing GID revisit at frame {frame_count}")
        revisit_global_ids(cam_id, active_tracks, inactive_tracks, frame_count)

    return frame

# --- بررسی طول ویدئوها و تنظیم رزولوشن ---
caps = [cv2.VideoCapture(path) for path in VIDEO_PATHS]
target_height = None
target_width = None
video_status = [True] * len(caps)
for i, cap in enumerate(caps):
    if not cap.isOpened():
        print(f"Error: Could not open video {VIDEO_PATHS[i]}")
        exit()
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video {i} resolution: {width}x{height}, frame count: {frame_count}")
    if target_height is None or target_width is None:
        target_height = height
        target_width = width
    else:
        target_height = min(target_height, height)
        target_width = min(target_width, width)

target_height = min(target_height, 720)
target_width = int(target_width * (target_height / target_height))
print(f"Target resolution for all frames: {target_width}x{target_height}")

# --- تنظیم ردیاب‌ها ---
trackers = [BYTETracker(args, frame_rate=30) for _ in range(len(VIDEO_PATHS))]

# --- حلقه اصلی ---
frame_counts = [0] * len(caps)
active_tracks = {0: {}, 1: {}}
out_videos = [cv2.VideoWriter(f"output_cam{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (target_width, target_height)) for i in range(len(caps))]
while any(video_status):
    for i, cap in enumerate(caps):
        if not video_status[i]:
            print(f"Cam {i} - Video already finished")
            continue
        
        ret, frame = cap.read()
        if not ret:
            print(f"Cam {i} - Finished processing video")
            video_status[i] = False
            continue
        
        frame_counts[i] += 1
        frame = process_frame(frame, trackers[i], cam_id=i, frame_count=frame_counts[i], 
                            active_tracks=active_tracks, inactive_tracks=inactive_tracks)
        frame = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_AREA)
        print(f"Cam {i} - Resized frame shape: {frame.shape}")
        cv2.imshow(f"MTMCT_Cam{i}", frame)
        out_videos[i].write(frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# آزادسازی منابع
for cap in caps:
    cap.release()
for out in out_videos:
    out.release()
cv2.destroyAllWindows()
print(f"Final frame counts: {frame_counts}")

# --- بررسی ذخیره‌سازی ویدیوهای خروجی ---
for i in range(len(out_videos)):
    output_path = f"output_cam{i}.mp4"
    if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
        print(f"Video for Cam {i} successfully saved at {output_path}")
    else:
        print(f"Error: Video for Cam {i} was not saved properly at {output_path}")

Video 0 resolution: 1920x1080, frame count: 561
Video 1 resolution: 1080x1920, frame count: 453
Target resolution for all frames: 1080x720
Number of raw detections: 0
No detections found after NMS
Cam 0 - YOLO detections shape: (0, 5)
Cam 0 - Number of tracked targets: 0
Cam 0 - Resized frame shape: (720, 1080, 3)
Number of raw detections: 0
No detections found after NMS
Cam 1 - YOLO detections shape: (0, 5)
Cam 1 - Number of tracked targets: 0
Cam 1 - Resized frame shape: (720, 1080, 3)
Number of raw detections: 0
No detections found after NMS
Cam 0 - YOLO detections shape: (0, 5)
Cam 0 - Number of tracked targets: 0
Cam 0 - Resized frame shape: (720, 1080, 3)
Number of raw detections: 0
No detections found after NMS
Cam 1 - YOLO detections shape: (0, 5)
Cam 1 - Number of tracked targets: 0
Cam 1 - Resized frame shape: (720, 1080, 3)
Number of raw detections: 0
No detections found after NMS
Cam 0 - YOLO detections shape: (0, 5)
Cam 0 - Number of tracked targets: 0
Cam 0 - Resized fram

KeyboardInterrupt: 

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from collections import deque, defaultdict
from scipy.optimize import linear_sum_assignment
from yolox.tracker.byte_tracker import BYTETracker
from torchvision import transforms, models
from ultralytics import YOLO
from types import SimpleNamespace
import warnings
warnings.filterwarnings('ignore')

# ============== مدل سیامی ==============
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet50(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.embedding = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 128),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = self.pool(x)
        x = self.embedding(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2

# ============== تنظیمات ==============
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIAMESE_MODEL_PATH = r"F:\mtmctwithsiamese\siameselast.pth"
YOLO_MODEL_PATH = "yolov8n.pt"
VIDEO_PATHS = [r"F:\mtmctwithsiamese\22.mp4", r"F:\mtmctwithsiamese\11.mp4"]

# پارامترهای بهینه‌شده
IMG_SIZE = (224, 224)
TRACKLET_MIN_LENGTH = 30  # حداقل طول tracklet برای matching
GALLERY_SIZE = 10  # تعداد بهترین frames در gallery
FEATURE_BUFFER_SIZE = 50  # تعداد features ذخیره شده
CONF_THRESH = 0.15
MIN_CONFIDENCE = 0.45  # افزایش threshold
MIN_BOX_SIZE = 50
IOU_THRESHOLD = 0.6
TRACK_BUFFER = 120

# Thresholds برای matching
DISTANCE_THRESH_STRICT = 0.35  # برای matching اولیه
DISTANCE_THRESH_LOOSE = 0.55  # برای inactive tracks
MIN_MATCH_CONFIDENCE = 0.7  # confidence مورد نیاز برای matching

# پارامترهای temporal
MAX_TIME_GAP = 180  # حداکثر فاصله زمانی (frames)
MIN_TEMPORAL_OVERLAP = 30  # حداقل overlap زمانی

# ============== پیش‌پردازش ==============
def get_transform():
    return transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# ============== BYTETracker Config ==============
args = SimpleNamespace(
    track_thresh=0.05,
    track_buffer=TRACK_BUFFER,
    match_thresh=0.85,
    min_box_area=MIN_BOX_SIZE * MIN_BOX_SIZE,
    mot20=False
)

# ============== بارگذاری مدل‌ها ==============
print(f"Loading Siamese model from {SIAMESE_MODEL_PATH}...")
model = SiameseNetwork().to(DEVICE)
try:
    checkpoint = torch.load(SIAMESE_MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print("✓ Siamese model loaded successfully")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find Siamese model at {SIAMESE_MODEL_PATH}")

print("Loading YOLO model...")
yolo_model = YOLO(YOLO_MODEL_PATH)
yolo_model.to(DEVICE)
print("✓ YOLO model loaded successfully")

# ============== کلاس Gallery برای مدیریت Features ==============
class FeatureGallery:
    """مدیریت یک gallery از بهترین features برای هر tracklet"""
    def __init__(self, max_size=GALLERY_SIZE):
        self.max_size = max_size
        self.features = []
        self.confidences = []
        self.images = deque(maxlen=FEATURE_BUFFER_SIZE)
        
    def add_feature(self, feature, confidence, image):
        """افزودن feature جدید با confidence وزن‌دهی شده"""
        self.features.append(feature)
        self.confidences.append(confidence)
        self.images.append(image)
        
        # نگه‌داشتن فقط بهترین features
        if len(self.features) > self.max_size:
            # مرتب‌سازی بر اساس confidence
            sorted_indices = np.argsort(self.confidences)[-self.max_size:]
            self.features = [self.features[i] for i in sorted_indices]
            self.confidences = [self.confidences[i] for i in sorted_indices]
    
    def get_aggregated_feature(self):
        """محاسبه feature تجمیعی با وزن‌دهی confidence"""
        if not self.features:
            return None
        
        # Normalize confidences
        confs = np.array(self.confidences)
        confs = confs / (confs.sum() + 1e-8)
        
        # Weighted average
        features_tensor = torch.stack(self.features)
        weights = torch.tensor(confs, dtype=torch.float32).to(DEVICE).unsqueeze(1)
        aggregated = (features_tensor * weights).sum(dim=0)
        
        return aggregated
    
    def get_best_images(self, n=5):
        """دریافت بهترین تصاویر"""
        if not self.images:
            return []
        
        # اگر gallery کمتر از n تصویر دارد، همه را برگردان
        if len(self.images) <= n:
            return list(self.images)
        
        # انتخاب n تصویر با بالاترین confidence
        sorted_indices = np.argsort(self.confidences)[-n:]
        return [list(self.images)[i] for i in sorted_indices if i < len(self.images)]

# ============== کلاس Tracklet ==============
class Tracklet:
    """نمایش یک tracklet با features، confidence و metadata"""
    def __init__(self, track_id, cam_id):
        self.track_id = track_id
        self.cam_id = cam_id
        self.gallery = FeatureGallery()
        self.start_frame = None
        self.end_frame = None
        self.global_id = None
        self.confidence_history = deque(maxlen=30)
        self.active = True
        
    def update(self, feature, confidence, image, frame_num):
        """به‌روزرسانی tracklet"""
        self.gallery.add_feature(feature, confidence, image)
        self.confidence_history.append(confidence)
        
        if self.start_frame is None:
            self.start_frame = frame_num
        self.end_frame = frame_num
    
    def get_avg_confidence(self):
        """محاسبه میانگین confidence"""
        if not self.confidence_history:
            return 0.0
        return np.mean(list(self.confidence_history))
    
    def get_length(self):
        """محاسبه طول tracklet"""
        if self.start_frame is None or self.end_frame is None:
            return 0
        return self.end_frame - self.start_frame + 1
    
    def is_valid_for_matching(self):
        """بررسی آیا tracklet برای matching مناسب است"""
        return (self.get_length() >= TRACKLET_MIN_LENGTH and 
                self.get_avg_confidence() >= MIN_MATCH_CONFIDENCE)

# ============== Global Registry ==============
class GlobalRegistry:
    """مدیریت global IDs و tracklets در تمام دوربین‌ها"""
    def __init__(self):
        self.global_id_counter = 0
        self.active_tracklets = defaultdict(dict)  # {cam_id: {track_id: Tracklet}}
        self.inactive_tracklets = defaultdict(dict)  # {cam_id: {track_id: Tracklet}}
        self.global_id_map = {}  # {global_id: [list of tracklets]}
        
    def get_new_global_id(self):
        """تخصیص یک global ID جدید"""
        new_id = self.global_id_counter
        self.global_id_counter += 1
        return new_id
    
    def add_tracklet(self, tracklet):
        """افزودن tracklet جدید"""
        self.active_tracklets[tracklet.cam_id][tracklet.track_id] = tracklet
    
    def deactivate_tracklet(self, cam_id, track_id, frame_num):
        """غیرفعال کردن tracklet"""
        if track_id in self.active_tracklets[cam_id]:
            tracklet = self.active_tracklets[cam_id].pop(track_id)
            tracklet.active = False
            tracklet.end_frame = frame_num
            self.inactive_tracklets[cam_id][track_id] = tracklet
            
    def get_all_tracklets_for_matching(self, current_cam_id):
        """دریافت تمام tracklets از دوربین‌های دیگر برای matching"""
        candidates = []
        
        for cam_id in self.active_tracklets:
            if cam_id != current_cam_id:
                for tracklet in self.active_tracklets[cam_id].values():
                    if tracklet.is_valid_for_matching():
                        candidates.append(tracklet)
        
        # اضافه کردن inactive tracklets
        for cam_id in self.inactive_tracklets:
            if cam_id != current_cam_id:
                for tracklet in self.inactive_tracklets[cam_id].values():
                    if tracklet.is_valid_for_matching():
                        candidates.append(tracklet)
        
        return candidates

# ============== محاسبه Feature ==============
def extract_feature(img, model):
    """استخراج feature از تصویر"""
    transform = get_transform()
    img_tensor = transform(img).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        feature = model.forward_once(img_tensor)
    
    return feature.squeeze(0)

def compute_distance(feature1, feature2):
    """محاسبه فاصله بین دو feature"""
    distance = torch.nn.functional.pairwise_distance(
        feature1.unsqueeze(0), 
        feature2.unsqueeze(0)
    ).item()
    return distance

# ============== Hungarian Algorithm Matching ==============
def compute_cost_matrix(query_tracklets, gallery_tracklets, current_frame):
    """محاسبه cost matrix بین tracklets"""
    n = len(query_tracklets)
    m = len(gallery_tracklets)
    
    if n == 0 or m == 0:
        return None, None, None
    
    cost_matrix = np.zeros((n, m))
    
    for i, query in enumerate(query_tracklets):
        query_feature = query.gallery.get_aggregated_feature()
        
        for j, gallery in enumerate(gallery_tracklets):
            gallery_feature = gallery.gallery.get_aggregated_feature()
            
            if query_feature is None or gallery_feature is None:
                cost_matrix[i, j] = 999.0  # خیلی بالا
                continue
            
            # محاسبه appearance distance
            appearance_dist = compute_distance(query_feature, gallery_feature)
            
            # Temporal constraint
            time_gap = abs(query.end_frame - gallery.end_frame)
            if time_gap > MAX_TIME_GAP:
                temporal_penalty = 999.0
            else:
                temporal_penalty = time_gap / MAX_TIME_GAP * 0.1
            
            # Confidence weighting
            conf_weight = (query.get_avg_confidence() + gallery.get_avg_confidence()) / 2
            
            # Final cost
            cost = appearance_dist + temporal_penalty
            cost = cost / (conf_weight + 0.1)  # کم کردن cost برای confidence بالا
            
            cost_matrix[i, j] = cost
    
    return cost_matrix, query_tracklets, gallery_tracklets

def match_tracklets(query_tracklets, gallery_tracklets, current_frame, strict=True):
    """Match tracklets using Hungarian algorithm"""
    cost_matrix, queries, galleries = compute_cost_matrix(
        query_tracklets, gallery_tracklets, current_frame
    )
    
    if cost_matrix is None:
        return []
    
    # Hungarian algorithm
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    matches = []
    threshold = DISTANCE_THRESH_STRICT if strict else DISTANCE_THRESH_LOOSE
    
    for i, j in zip(row_ind, col_ind):
        if cost_matrix[i, j] < threshold:
            matches.append({
                'query': queries[i],
                'gallery': galleries[j],
                'distance': cost_matrix[i, j],
                'confidence': 1.0 - (cost_matrix[i, j] / threshold)
            })
    
    return matches

# ============== YOLO Detection ==============
def get_yolo_detections(frame):
    """دریافت detections از YOLO"""
    results = yolo_model.predict(frame, conf=CONF_THRESH, classes=[0], verbose=False)
    detections = []
    
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        
        # NMS
        indices = cv2.dnn.NMSBoxes(
            boxes.tolist(), confs.tolist(), 
            CONF_THRESH, IOU_THRESHOLD
        )
        
        if len(indices) > 0:
            indices = indices.flatten()
            for i in indices:
                box = boxes[i]
                conf = confs[i]
                
                # فیلتر کردن boxes کوچک و confidence پایین
                if (box[2] - box[0]) >= MIN_BOX_SIZE and \
                   (box[3] - box[1]) >= MIN_BOX_SIZE and \
                   conf >= MIN_CONFIDENCE:
                    detections.append(np.concatenate([box, [conf]]))
    
    return np.array(detections) if detections else np.empty((0, 5))

# ============== Process Frame ==============
def process_frame(frame, tracker, cam_id, frame_count, registry, model):
    """پردازش یک frame"""
    detections = get_yolo_detections(frame)
    
    # Tracking
    img_info = [frame.shape[0], frame.shape[1]]
    if detections.shape[0] > 0:
        online_targets = tracker.update(detections, img_info, frame.shape[:2])
    else:
        online_targets = []
    
    current_track_ids = set()
    
    # پردازش هر target
    for t in online_targets:
        tid = t.track_id
        current_track_ids.add(tid)
        
        # Crop person image
        tlbr = t.tlbr.astype(int)
        person_img = frame[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]]
        
        # بررسی validity
        if person_img.size == 0 or \
           person_img.shape[0] < MIN_BOX_SIZE or \
           person_img.shape[1] < MIN_BOX_SIZE:
            continue
        
        confidence = t.score if hasattr(t, 'score') else 0.0
        
        # ایجاد tracklet جدید یا به‌روزرسانی
        if tid not in registry.active_tracklets[cam_id]:
            tracklet = Tracklet(tid, cam_id)
            registry.add_tracklet(tracklet)
        else:
            tracklet = registry.active_tracklets[cam_id][tid]
        
        # استخراج feature و به‌روزرسانی tracklet
        feature = extract_feature(person_img, model)
        tracklet.update(feature, confidence, person_img, frame_count)
        
        # اگر tracklet آماده matching است و هنوز global ID ندارد
        if tracklet.global_id is None and tracklet.is_valid_for_matching():
            # سعی در matching با tracklets موجود
            candidates = registry.get_all_tracklets_for_matching(cam_id)
            
            if candidates:
                matches = match_tracklets([tracklet], candidates, frame_count, strict=True)
                
                if matches and len(matches) > 0:
                    best_match = matches[0]
                    matched_tracklet = best_match['gallery']
                    
                    # تخصیص global ID از matched tracklet
                    if matched_tracklet.global_id is not None:
                        tracklet.global_id = matched_tracklet.global_id
                        print(f"Cam {cam_id} Track {tid} matched with GID {tracklet.global_id} "
                              f"(distance: {best_match['distance']:.3f})")
        
        # اگر هنوز global ID ندارد، یکی جدید بده
        if tracklet.global_id is None and tracklet.get_length() >= TRACKLET_MIN_LENGTH:
            tracklet.global_id = registry.get_new_global_id()
            print(f"Cam {cam_id} Track {tid} assigned new GID {tracklet.global_id}")
        
        # رسم bounding box
        global_id = tracklet.global_id
        label = f"GID:{global_id}" if global_id is not None else f"TID:{tid}"
        color = (0, 255, 0) if global_id is not None else (255, 255, 0)
        
        cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), color, 2)
        cv2.putText(frame, label, (tlbr[0], tlbr[1] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # غیرفعال کردن tracklets که دیگر دیده نمی‌شوند
    for tid in list(registry.active_tracklets[cam_id].keys()):
        if tid not in current_track_ids:
            registry.deactivate_tracklet(cam_id, tid, frame_count)
    
    return frame

# ============== Main Loop ==============
def main():
    print("\n" + "="*60)
    print("MTMCT Advanced System - Starting...")
    print("="*60 + "\n")
    
    # ایجاد registry
    registry = GlobalRegistry()
    
    # باز کردن ویدیوها
    caps = [cv2.VideoCapture(path) for path in VIDEO_PATHS]
    trackers = [BYTETracker(args, frame_rate=30) for _ in range(len(VIDEO_PATHS))]
    
    # تنظیم resolution
    target_height = 720
    target_width = 1280
    
    # ایجاد video writers
    out_videos = [
        cv2.VideoWriter(
            f"output_cam{i}.mp4",
            cv2.VideoWriter_fourcc(*'mp4v'),
            30,
            (target_width, target_height)
        )
        for i in range(len(caps))
    ]
    
    frame_counts = [0] * len(caps)
    video_status = [True] * len(caps)
    
    print("Processing videos...")
    
    while any(video_status):
        for i, cap in enumerate(caps):
            if not video_status[i]:
                continue
            
            ret, frame = cap.read()
            if not ret:
                video_status[i] = False
                print(f"Cam {i} finished")
                continue
            
            frame_counts[i] += 1
            
            # پردازش frame
            processed_frame = process_frame(
                frame, trackers[i], i, frame_counts[i], registry, model
            )
            
            # Resize و ذخیره
            processed_frame = cv2.resize(
                processed_frame, 
                (target_width, target_height),
                interpolation=cv2.INTER_AREA
            )
            
            cv2.imshow(f"MTMCT_Cam{i}", processed_frame)
            out_videos[i].write(processed_frame)
            
            # نمایش آمار
            if frame_counts[i] % 100 == 0:
                active = len(registry.active_tracklets[i])
                inactive = len(registry.inactive_tracklets[i])
                print(f"Cam {i} - Frame {frame_counts[i]}: "
                      f"Active={active}, Inactive={inactive}, "
                      f"Total GIDs={registry.global_id_counter}")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # آزادسازی منابع
    for cap in caps:
        cap.release()
    for out in out_videos:
        out.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*60)
    print("Processing complete!")
    print(f"Total Global IDs created: {registry.global_id_counter}")
    print("="*60 + "\n")

if __name__ == "__main__":
    main()

Loading Siamese model from F:\mtmctwithsiamese\siameselast.pth...
✓ Siamese model loaded successfully
Loading YOLO model...
✓ YOLO model loaded successfully

MTMCT Advanced System - Starting...

Processing videos...
Cam 0 Track 12 assigned new GID 0
Cam 0 Track 13 assigned new GID 1
Cam 0 - Frame 100: Active=2, Inactive=1, Total GIDs=2
Cam 1 Track 14 assigned new GID 2
Cam 1 - Frame 100: Active=2, Inactive=0, Total GIDs=3
Cam 1 Track 15 assigned new GID 3
Cam 1 Track 16 assigned new GID 4
Cam 0 Track 17 assigned new GID 5
Cam 1 Track 18 assigned new GID 6
Cam 0 - Frame 200: Active=3, Inactive=1, Total GIDs=7
Cam 1 - Frame 200: Active=2, Inactive=2, Total GIDs=7


KeyboardInterrupt: 

DISTANCE_THRESH (مقدار فعلی: 0.5):
کارکرد: این آستانه تعیین می‌کنه که فاصله (distance) محاسبه‌شده توسط مدل سیامی بین فریم‌های یک فرد جدید و فریم‌های افراد فعال در دوربین دیگر چقدر باید باشه تا بهشون یک GID یکسان تخصیص داده بشه.
تأثیر:
کاهش (مثلاً به 0.3): سخت‌گیرانه‌تر می‌شه، یعنی فقط وقتی فریم‌ها خیلی شبیه باشن (فاصله کم) GID یکسان می‌گیرن. این می‌تونه از تخصیص اشتباه GID به افراد متفاوت جلوگیری کنه، اما ممکنه باعث بشه افراد یکسان در دوربین‌های مختلف GIDهای جداگونه بگیرن (مشکل تطبیق نشدن).
افزایش (مثلاً به 0.7): انعطاف‌پذیرتر می‌شه، یعنی حتی اگر فریم‌ها کمی متفاوت باشن، ممکنه GID یکسان بگیرن. این می‌تونه تطبیق افراد یکسان بین دوربین‌ها رو بهبود بده، اما خطر تخصیص GID یکسان به افراد متفاوت رو افزایش می‌ده.
مشکل مرتبط:
اگر افراد متفاوت در یک دوربین GID یکسان می‌گیرن: کاهش بدید (مثلاً به 0.4 یا 0.3).
اگر افراد یکسان در دوربین‌های مختلف GIDهای متفاوت می‌گیرن: کمی افزایش بدید (مثلاً به 0.6).
INACTIVE_DISTANCE_THRESH (مقدار فعلی: 0.7):
کارکرد: این آستانه برای مقایسه فریم‌های یک فرد جدید با فریم‌های افراد غیرفعال (inactive tracks) در دوربین دیگر استفاده می‌شه. اگر فاصله کمتر از این مقدار باشه، GID فرد غیرفعال به فرد جدید تخصیص داده می‌شه.
تأثیر:
کاهش (مثلاً به 0.5): سخت‌گیرانه‌تر می‌شه، یعنی فقط وقتی فریم‌ها خیلی شبیه باشن، GID غیرفعال استفاده می‌شه. این می‌تونه از تخصیص اشتباه جلوگیری کنه، اما ممکنه باعث بشه افراد یکسان GID جدید بگیرن.
افزایش (مثلاً به 0.9): انعطاف‌پذیرتر می‌شه، یعنی حتی با تفاوت‌های بیشتر، GID غیرفعال تخصیص داده می‌شه. این می‌تونه تطبیق افراد یکسان رو بهبود بده، اما خطر تخصیص اشتباه رو افزایش می‌ده.
مشکل مرتبط:
اگر افراد متفاوت به اشتباه GID غیرفعال می‌گیرن: کاهش بدید (مثلاً به 0.6).
اگر افراد یکسان با tracks غیرفعال تطبیق داده نمی‌شن: افزایش بدید (مثلاً به 0.8).
MIN_CONFIDENCE (مقدار فعلی: 0.4):
کارکرد: حداقل اطمینان (confidence score) تشخیص‌های YOLO برای پذیرش یک جعبه (bounding box) به‌عنوان یک فرد. فقط جعبه‌هایی با اطمینان بیشتر از این مقدار برای ردیابی و تخصیص GID استفاده می‌شن.
تأثیر:
کاهش (مثلاً به 0.3): جعبه‌های بیشتری (حتی با اطمینان کمتر) پذیرفته می‌شن، که می‌تونه تعداد تشخیص‌ها رو افزایش بده، اما ممکنه جعبه‌های نادرست یا نویزدار هم وارد پروسه بشن.
افزایش (مثلاً به 0.5): فقط جعبه‌های با اطمینان بالا پذیرفته می‌شن، که می‌تونه کیفیت تشخیص‌ها رو بهبود بده، اما ممکنه افراد با جعبه‌های کم‌کیفیت (مثلاً در نور کم) از دست برن.
مشکل مرتبط:
اگر افراد واقعی تشخیص داده نمی‌شن: کاهش بدید (مثلاً به 0.3).
اگر جعبه‌های نادرست (مثلاً اشیاء غیرانسانی) ردیابی می‌شن: افزایش بدید (مثلاً به 0.5).
CONF_THRESH (مقدار فعلی: 0.15):
کارکرد: حداقل اطمینان اولیه برای تشخیص‌های YOLO قبل از اعمال NMS (Non-Maximum Suppression). این مقدار تعیین می‌کنه که کدوم جعبه‌ها وارد مرحله فیلتر NMS بشن.
تأثیر:
کاهش (مثلاً به 0.1): جعبه‌های بیشتری وارد پروسه می‌شن، که می‌تونه تشخیص افراد در شرایط سخت (مثلاً نور کم) رو بهبود بده، اما نویز رو افزایش می‌ده.
افزایش (مثلاً به 0.2): فقط جعبه‌های با اطمینان بالاتر وارد می‌شن، که نویز رو کاهش می‌ده، اما ممکنه افراد با تشخیص ضعیف حذف بشن.
مشکل مرتبط:
اگر افراد در ویدیوهای کم‌کیفیت تشخیص داده نمی‌شن: کاهش بدید (مثلاً به 0.1).
اگر تعداد زیادی جعبه نادرست دارید: افزایش بدید (مثلاً به 0.2).
IOU_THRESHOLD (مقدار فعلی: 0.6):
کارکرد: آستانه برای NMS در YOLO. این مقدار تعیین می‌کنه که جعبه‌های همپوشان (overlapping) چقدر باید مشابه باشن تا یکی‌شون حذف بشه.
تأثیر:
کاهش (مثلاً به 0.5): NMS انعطاف‌پذیرتر می‌شه و جعبه‌های بیشتری حفظ می‌شن، که می‌تونه در ویدیوهای شلوغ کمک کنه، اما ممکنه جعبه‌های تکراری بمونن.
افزایش (مثلاً به 0.7): NMS سخت‌گیرانه‌تر می‌شه و جعبه‌های مشابه حذف می‌شن، که می‌تونه نویز رو کاهش بده، اما ممکنه جعبه‌های معتبر حذف بشن.
مشکل مرتبط:
اگر جعبه‌های تکراری برای یک فرد ظاهر می‌شن: افزایش بدید (مثلاً به 0.7).
اگر افراد در ویدیوهای شلوغ از دست می‌رن: کاهش بدید (مثلاً به 0.5).
MIN_BOX_SIZE (مقدار فعلی: 40):
کارکرد: حداقل اندازه جعبه (عرض و ارتفاع) برای پذیرش یک تشخیص به‌عنوان فرد. جعبه‌های کوچک‌تر از این مقدار نادیده گرفته می‌شن.
تأثیر:
کاهش (مثلاً به 30): جعبه‌های کوچک‌تر پذیرفته می‌شن، که می‌تونه برای تشخیص افراد دور یا کوچک مفید باشه، اما نویز (مثلاً اشیاء کوچک) رو افزایش می‌ده.
افزایش (مثلاً به 50): فقط جعبه‌های بزرگ‌تر پذیرفته می‌شن، که نویز رو کاهش می‌ده، اما ممکنه افراد کوچک یا دور حذف بشن.
مشکل مرتبط:
اگر افراد کوچک یا دور تشخیص داده نمی‌شن: کاهش بدید (مثلاً به 30).
اگر اشیاء کوچک به اشتباه ردیابی می‌شن: افزایش بدید (مثلاً به 50).
FRAME_BUFFER (مقدار فعلی: 20):
کارکرد: تعداد فریم‌هایی که برای هر track ذخیره می‌شن تا برای تخصیص GID استفاده بشن. وقتی تعداد فریم‌های معتبر به این مقدار برسه و حداقل 70٪ فریم‌ها معتبر باشن، GID تخصیص داده می‌شه.
تأثیر:
کاهش (مثلاً به 15): GID زودتر تخصیص داده می‌شه، که می‌تونه برای ویدیوهای کوتاه یا سریع مفید باشه، اما ممکنه با فریم‌های کمتر، تخصیص نادرست بشه.
افزایش (مثلاً به 30): تخصیص GID با فریم‌های بیشتری انجام می‌شه، که دقت رو افزایش می‌ده، اما تأخیر در تخصیص ایجاد می‌کنه.
مشکل مرتبط:
اگر GIDها خیلی دیر تخصیص داده می‌شن: کاهش بدید (مثلاً به 15).
اگر GIDها با داده‌های ناکافی تخصیص داده می‌شن: افزایش بدید (مثلاً به 25).
DUPLICATE_GID_THRESHOLD (مقدار فعلی: 5):
کارکرد: تعداد فریم‌هایی که اگر دو نفر در یک دوربین GID یکسان داشته باشن، عملیات بازبینی (revisit) انجام می‌شه.
تأثیر:
کاهش (مثلاً به 3): بازبینی زودتر انجام می‌شه، که می‌تونه تخصیص‌های نادرست رو سریع‌تر اصلاح کنه، اما ممکنه بازبینی‌های غیرضروری انجام بشه.
افزایش (مثلاً به 10): بازبینی دیرتر انجام می‌شه، که از بازبینی‌های غیرضروری جلوگیری می‌کنه، اما ممکنه GIDهای نادرست مدت بیشتری بمونن.
مشکل مرتبط:
اگر GIDهای تکراری دیر اصلاح می‌شن: کاهش بدید (مثلاً به 3).
اگر بازبینی‌های غیرضروری زیادن: افزایش بدید (مثلاً به 7).
REVISIT_INTERVAL (مقدار فعلی: 100):
کارکرد: تعداد فریم‌هایی که بعد از اون بازبینی دوره‌ای GIDها انجام می‌شه.
تأثیر:
کاهش (مثلاً به 50): بازبینی مکررتر انجام می‌شه، که می‌تونه GIDهای نادرست رو سریع‌تر اصلاح کنه، اما بار محاسباتی رو افزایش می‌ده.
افزایش (مثلاً به 200): بازبینی کمتر انجام می‌شه، که بار محاسباتی رو کاهش می‌ده، اما ممکنه GIDهای نادرست مدت بیشتری بمونن.
مشکل مرتبط:
اگر GIDهای نادرست دیر اصلاح می‌شن: کاهش بدید (مثلاً به 50).
اگر عملکرد کند است: افزایش بدید (مثلاً به 150).
REVISIT_COOLDOWN (مقدار فعلی: 30):
کارکرد: حداقل تعداد فریم‌هایی که باید بین دو بازبینی برای یک GID فاصله باشه.
تأثیر:
کاهش (مثلاً به 20): بازبینی‌ها می‌تونن سریع‌تر تکرار بشن، که برای اصلاح سریع‌تر GIDهای نادرست خوبه، اما ممکنه باعث ناپایداری بشه.
افزایش (مثلاً به 50): بازبینی‌ها کمتر تکرار می‌شن، که پایداری رو افزایش می‌ده، اما ممکنه اصلاح GIDهای نادرست دیرتر انجام بشه.
مشکل مرتبط:
اگر GIDهای نادرست دیر اصلاح می‌شن: کاهش بدید (مثلاً به 20).
اگر بازبینی‌ها باعث ناپایداری می‌شن: افزایش بدید (مثلاً به 40).
NUM_COMPARISON_IMAGES (مقدار فعلی: 3):
کارکرد: تعداد فریم‌هایی که برای محاسبه میانگین فاصله در مدل سیامی استفاده می‌شن.
تأثیر:
کاهش (مثلاً به 2): میانگین فاصله با فریم‌های کمتری محاسبه می‌شه، که می‌تونه سرعت رو افزایش بده، اما دقت رو کاهش می‌ده.
افزایش (مثلاً به 5): میانگین فاصله با فریم‌های بیشتری محاسبه می‌شه، که دقت رو افزایش می‌ده، اما بار محاسباتی رو بیشتر می‌کنه.
مشکل مرتبط:
اگر تخصیص GIDها نادرست است: افزایش بدید (مثلاً به 4).
اگر پردازش کند است: کاهش بدید (مثلاً به 2).
تنظیمات BYTETracker (track_thresh, match_thresh, track_buffer):
track_thresh (مقدار فعلی: 0.05):
کارکرد: حداقل اطمینان برای شروع یک track جدید توسط BYTETracker.
تأثیر:
کاهش: trackهای بیشتری شروع می‌شن، که می‌تونه افراد بیشتری رو ردیابی کنه، اما نویز رو افزایش می‌ده.
افزایش: فقط trackهای با اطمینان بالا شروع می‌شن، که نویز رو کاهش می‌ده، اما ممکنه افراد با تشخیص ضعیف از دست برن.
مشکل مرتبط:
اگر افراد واقعی ردیابی نمی‌شن: کاهش بدید (مثلاً به 0.03).
اگر trackهای نادرست زیادن: افزایش بدید (مثلاً به 0.1).
match_thresh (مقدار فعلی: 0.8):
کارکرد: آستانه برای تطبیق جعبه‌های جدید با trackهای موجود.
تأثیر:
کاهش: تطبیق انعطاف‌پذیرتر می‌شه، که می‌تونه پایداری trackها رو در شرایط شلوغ بهبود بده، اما ممکنه تطبیق‌های نادرست افزایش پیدا کنه.
افزایش: تطبیق سخت‌گیرانه‌تر می‌شه، که دقت رو افزایش می‌ده، اما ممکنه trackها گم بشن.
مشکل مرتبط:
اگر trackها گم می‌شن یا IDها عوض می‌شن: کاهش بدید (مثلاً به 0.7).
اگر تطبیق‌های نادرست زیادن: افزایش بدید (مثلاً به 0.9).
track_buffer (مقدار فعلی: 120):
کارکرد: تعداد فریم‌هایی که یک track غیرفعال نگه داشته می‌شه قبل از حذف.
تأثیر:
کاهش: trackهای غیرفعال زودتر حذف می‌شن، که حافظه رو آزاد می‌کنه، اما ممکنه افراد موقتاً گم‌شده دوباره ردیابی نشن.
افزایش: trackهای غیرفعال مدت بیشتری نگه داشته می‌شن، که می‌تونه برای افراد موقتاً گم‌شده مفید باشه، اما حافظه بیشتری مصرف می‌کنه.
مشکل مرتبط:
اگر افراد موقتاً گم‌شده دوباره ردیابی نمی‌شن: افزایش بدید (مثلاً به 150).
اگر حافظه یا عملکرد مشکل داره: کاهش بدید (مثلاً به 100).

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from collections import deque
from yolox.tracker.byte_tracker import BYTETracker
from torchvision import transforms, models
from ultralytics import YOLO
from types import SimpleNamespace

# --- تعریف مدل سیامی (بدون تغییر) ---
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet50(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.embedding = nn.Sequential(nn.Flatten(), nn.Linear(2048, 512), nn.ReLU(), nn.BatchNorm1d(512), nn.Linear(512, 128), nn.Sigmoid())
    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = self.pool(x)
        x = self.embedding(x)
        return x
    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)

# --- CONFIG ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIAMESE_MODEL_PATH = r"F:\mtmctwithsiamese\siameselast.pth"
YOLO_MODEL_PATH = "yolov8n.pt"
VIDEO_PATHS = [r"F:\mtmctwithsiamese\22.mp4", r"F:\mtmctwithsiamese\11.mp4"]
IMG_SIZE = (224, 224)
CONF_THRESH = 0.3
MIN_BOX_SIZE = 40
IOU_THRESHOLD = 0.7

# --- هایپرپارامترهای کلیدی جدید ---
GALLERY_MAX_SIZE = 30
# آستانه سخت‌گیرانه‌تر، چون مقایسه گالری به گالری دقیق‌تر است
GID_MATCH_THRESHOLD = 0.55
# تعداد فریم‌هایی که منتظر می‌مانیم تا برای یک فرد ناشناس شواهد جمع کنیم
CONFIRMATION_FRAME_THRESHOLD = 25
# برای مقایسه، چند تصویر از هر گالری استفاده شود
NUM_GALLERY_COMPARISONS = 20

# --- تنظیمات BYTETracker (بدون تغییر) ---
args = SimpleNamespace(track_thresh=0.5, track_buffer=120, match_thresh=0.8, min_box_area=MIN_BOX_SIZE*MIN_BOX_SIZE, mot20=False)

# --- بارگذاری مدل‌ها (بدون تغییر) ---
print("Loading models...")
siamese_model = SiameseNetwork().to(DEVICE)
checkpoint = torch.load(SIAMESE_MODEL_PATH, map_location=DEVICE)
siamese_model.load_state_dict(checkpoint['model_state_dict'])
siamese_model.eval()
yolo_model = YOLO(YOLO_MODEL_PATH).to(DEVICE)
print("Models loaded successfully.")

# --- ساختارهای داده اصلی (با یک ساختار جدید) ---
global_id_counter = 0
# گالری مرکزی برای هویت‌های تایید شده
global_person_gallery = {} # {gid: {"image_gallery": deque, ...}}
# نگاشت TID موقت به GID تایید شده در هر دوربین
active_tracks = {i: {} for i in range(len(VIDEO_PATHS))} # {cam_id: {tid: gid}}
# ساختار داده جدید برای ترک‌های ناشناس که در حال جمع‌آوری شواهد هستند
unidentified_tracks = {i: {} for i in range(len(VIDEO_PATHS))} # {cam_id: {tid: {"image_gallery": deque, "first_seen_frame": int, ...}}}


# --- توابع کمکی (با یک تابع جدید) ---
def preprocess_image(img):
    transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize(IMG_SIZE), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    return transform(img)

def compute_siamese_distance(img1, img2, model):
    img1_tensor = preprocess_image(img1).unsqueeze(0).to(DEVICE)
    img2_tensor = preprocess_image(img2).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        emb1, emb2 = model(img1_tensor, img2_tensor)
        return torch.nn.functional.pairwise_distance(emb1, emb2).item()

def get_yolo_detections(frame):
    results = yolo_model.predict(frame, conf=CONF_THRESH, classes=[0], verbose=False)
    detections = []
    for result in results:
        boxes, confs = result.boxes.xyxy.cpu().numpy(), result.boxes.conf.cpu().numpy()
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confs.tolist(), CONF_THRESH, IOU_THRESHOLD)
        if len(indices) > 0:
            for i in indices.flatten():
                box = boxes[i]
                if (box[2] - box[0]) >= MIN_BOX_SIZE and (box[3] - box[1]) >= MIN_BOX_SIZE:
                    detections.append(np.concatenate([box, [confs[i]]]))
    return np.array(detections) if detections else np.empty((0, 5))

def compare_galleries(gallery1, gallery2):
    """دو گالری تصویر را با هم مقایسه کرده و میانگین فاصله را برمی‌گرداند."""
    distances = []
    num_g1 = min(NUM_GALLERY_COMPARISONS, len(gallery1))
    num_g2 = min(NUM_GALLERY_COMPARISONS, len(gallery2))
    
    # مقایسه متقاطع بین تصاویر دو گالری
    for img1 in list(gallery1)[-num_g1:]:
        for img2 in list(gallery2)[-num_g2:]:
            dist = compute_siamese_distance(img1, img2, siamese_model)
            distances.append(dist)
    
    return np.mean(distances) if distances else float('inf')

def find_best_match_for_gallery(unidentified_gallery, current_cam_id):
    """گالری یک فرد ناشناس را با گالری تمام افراد تایید شده مقایسه می‌کند."""
    min_avg_dist = float('inf')
    best_gid = -1
    
    active_gids_in_current_cam = set(active_tracks[current_cam_id].values())

    for gid, data in global_person_gallery.items():
        if gid in active_gids_in_current_cam:
            continue
        
        dist = compare_galleries(unidentified_gallery, data["image_gallery"])
        if dist < min_avg_dist:
            min_avg_dist = dist
            best_gid = gid
            
    if min_avg_dist < GID_MATCH_THRESHOLD:
        return best_gid, min_avg_dist
    else:
        return -1, min_avg_dist


# --- تابع اصلی پردازش فریم (بازنویسی شده) ---
def process_frame(frame, tracker, cam_id, frame_count):
    global global_id_counter

    detections = get_yolo_detections(frame)
    online_targets = tracker.update(detections, [frame.shape[0], frame.shape[1]], frame.shape[:2])
    
    current_frame_tids = set()
    
    # پردازش ترک‌های آنلاین
    for t in online_targets:
        tid, tlbr = t.track_id, t.tlbr.astype(int)
        current_frame_tids.add(tid)
        person_img = frame[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]]
        if person_img.size == 0: continue

        label, color = f"ID:?", (255, 165, 0) # رنگ نارنجی برای ناشناس

        # Case 1: ترک از قبل شناسایی شده و فعال است
        if tid in active_tracks[cam_id]:
            gid = active_tracks[cam_id][tid]
            global_person_gallery[gid]["image_gallery"].append(person_img)
            global_person_gallery[gid]["last_seen_cam"] = cam_id
            global_person_gallery[gid]["last_seen_frame"] = frame_count
            label, color = f"ID:{gid}", (0, 255, 0) # سبز برای تایید شده

        # Case 2: ترک در لیست ناشناس‌ها قرار دارد و در حال جمع‌آوری شواهد است
        elif tid in unidentified_tracks[cam_id]:
            track_info = unidentified_tracks[cam_id][tid]
            track_info["image_gallery"].append(person_img)
            track_info["last_bbox"] = tlbr
            
            # آیا به اندازه کافی شواهد جمع شده تا هویت را تایید کنیم؟
            if frame_count - track_info["first_seen_frame"] > CONFIRMATION_FRAME_THRESHOLD:
                print(f"Cam {cam_id} | TID {tid}: Confirmation phase started...")
                best_gid, dist = find_best_match_for_gallery(track_info["image_gallery"], cam_id)
                
                # Sub-Case 2a: تطابق پیدا شد -> ترفیع به GID موجود
                if best_gid != -1:
                    gid = best_gid
                    active_tracks[cam_id][tid] = gid
                    # ادغام گالری موقت در گالری اصلی
                    for img in track_info["image_gallery"]:
                        global_person_gallery[gid]["image_gallery"].append(img)
                    del unidentified_tracks[cam_id][tid]
                    label, color = f"ID:{gid}", (0, 255, 0)
                    print(f"Cam {cam_id} | TID {tid}: Promoted to GID {gid} (Dist: {dist:.2f})")

                # Sub-Case 2b: تطابق پیدا نشد -> ترفیع به GID جدید
                else:
                    new_gid = global_id_counter
                    global_id_counter += 1
                    gid = new_gid
                    active_tracks[cam_id][tid] = gid
                    global_person_gallery[gid] = {
                        "image_gallery": track_info["image_gallery"],
                        "last_seen_cam": cam_id,
                        "last_seen_frame": frame_count
                    }
                    del unidentified_tracks[cam_id][tid]
                    label, color = f"ID:{gid}", (0, 255, 0)
                    print(f"Cam {cam_id} | TID {tid}: Promoted to NEW GID {gid}")
            
        # Case 3: ترک کاملاً جدید است -> اضافه به لیست ناشناس‌ها
        else:
            unidentified_tracks[cam_id][tid] = {
                "image_gallery": deque([person_img], maxlen=GALLERY_MAX_SIZE),
                "first_seen_frame": frame_count,
                "last_bbox": tlbr
            }
            print(f"Cam {cam_id} | New unidentified TID {tid} appeared.")

        # رسم bounding box بر اساس وضعیت فعلی
        final_bbox = tlbr
        if tid in active_tracks[cam_id]:
            gid = active_tracks[cam_id][tid]
            label, color = f"ID:{gid}", (0, 255, 0)
        elif tid in unidentified_tracks[cam_id]:
            final_bbox = unidentified_tracks[cam_id][tid]["last_bbox"]
            label, color = f"ID:?", (255, 165, 0)
            
        cv2.rectangle(frame, (final_bbox[0], final_bbox[1]), (final_bbox[2], final_bbox[3]), color, 2)
        cv2.putText(frame, label, (final_bbox[0], final_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
    # --- مدیریت ترک‌های از دست رفته ---
    # حذف از هر دو لیست فعال و ناشناس
    lost_tids = set(active_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_tids: del active_tracks[cam_id][tid]
        
    lost_unidentified_tids = set(unidentified_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_unidentified_tids: del unidentified_tracks[cam_id][tid]
        
    return frame

# --- حلقه اصلی پردازش ویدیو (بدون تغییر) ---
caps = [cv2.VideoCapture(path) for path in VIDEO_PATHS]
# یک رزولوشن استاندارد برای نمایش هر دو ویدیو تعیین می‌کنیم
w, h = 960, 540 
trackers = [BYTETracker(args, frame_rate=30) for _ in range(len(VIDEO_PATHS))]
frame_counts = [0] * len(caps)
out_videos = [cv2.VideoWriter(f"output_cam{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 25, (w, h)) for i in range(len(caps))]

while True:
    frames = []
    rets = []
    # 1. خواندن یک فریم از هر ویدیو
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame)

    # اگر یکی از ویدیوها تمام شد، حلقه را متوقف کن
    if not all(rets):
        print("Finished processing one of the videos.")
        break

    processed_frames = []
    # 2. پردازش فریم مربوط به هر دوربین
    for i, frame in enumerate(frames):
        if frame is None:
            continue
        
        frame_counts[i] += 1
        # فراخوانی تابع اصلی پردازشگر
        processed_frame = process_frame(frame, trackers[i], i, frame_counts[i])
        
        # تغییر اندازه فریم پردازش شده به سایز استاندارد
        resized_frame = cv2.resize(processed_frame, (w, h))
        
        # اضافه کردن فریم پردازش شده به لیست
        processed_frames.append(resized_frame)
        
        # ذخیره فریم در فایل ویدیویی خروجی
        out_videos[i].write(resized_frame)

    # 3. ترکیب فریم‌ها و نمایش
    # این شرط برای جلوگیری از خطا در صورت خالی بودن لیست است
    if len(processed_frames) == len(caps):
        # **کلید اصلی**: فریم‌ها را به صورت افقی به هم می‌چسبانیم
        combined_frame = cv2.hconcat(processed_frames)
        
        # نمایش تصویر واحد و ترکیبی
        cv2.imshow("Multi-Camera Live Tracking", combined_frame)
    
    # شرط خروج با زدن کلید 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# آزادسازی منابع
print("Releasing resources...")
for cap in caps:
    cap.release()
for out in out_videos:
    out.release()
cv2.destroyAllWindows()
print("All done.")


Loading models...
Models loaded successfully.
Cam 0 | New unidentified TID 17 appeared.
Cam 0 | TID 17: Confirmation phase started...
Cam 0 | TID 17: Promoted to NEW GID 0
Cam 0 | New unidentified TID 18 appeared.
Cam 1 | New unidentified TID 19 appeared.
Cam 1 | New unidentified TID 20 appeared.
Cam 0 | TID 18: Confirmation phase started...
Cam 0 | TID 18: Promoted to NEW GID 1
Cam 1 | TID 19: Confirmation phase started...


KeyboardInterrupt: 

In [1]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from collections import deque
from yolox.tracker.byte_tracker import BYTETracker
from torchvision import transforms, models
from ultralytics import YOLO
from types import SimpleNamespace

# --- تعریف مدل سیامی (بدون تغییر) ---
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet50(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.embedding = nn.Sequential(nn.Flatten(), nn.Linear(2048, 512), nn.ReLU(), nn.BatchNorm1d(512), nn.Linear(512, 128), nn.Sigmoid())
    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = self.pool(x)
        x = self.embedding(x)
        return x
    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)

# --- CONFIG ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIAMESE_MODEL_PATH = r"F:\mtmctwithsiamese\siameselast.pth"
YOLO_MODEL_PATH = "yolov8n.pt"
VIDEO_PATHS = [r"F:\mtmctwithsiamese\22.mp4", r"F:\mtmctwithsiamese\11.mp4"]
IMG_SIZE = (224, 224)
CONF_THRESH = 0.3
MIN_BOX_SIZE = 40
IOU_THRESHOLD = 0.7

# --- هایپرپارامترهای کلیدی و منعطف ---
# افزایش ظرفیت گالری برای داشتن تاریخچه غنی‌تر از ظاهر هر فرد
GALLERY_MAX_SIZE = 50
# آستانه تطابق، نه خیلی سخت‌گیرانه و نه خیلی آسان‌گیر
GID_MATCH_THRESHOLD = 0.48
# حدود یک ثانیه (با فریم ریت 25) صبر می‌کنیم تا شواهد کافی جمع شود
CONFIRMATION_FRAME_THRESHOLD = 25
# تعداد ویژگی‌های مورد استفاده برای مقایسه برای افزایش سرعت
NUM_EMBEDDINGS_COMPARISONS = 20

# --- تنظیمات BYTETracker (نقش آن محدود به پیش‌بینی حرکت است) ---
args = SimpleNamespace(track_thresh=0.5, track_buffer=90, match_thresh=0.8, min_box_area=MIN_BOX_SIZE*MIN_BOX_SIZE, mot20=False)

# --- بارگذاری مدل‌ها ---
print("Loading models...")
siamese_model = SiameseNetwork().to(DEVICE)
checkpoint = torch.load(SIAMESE_MODEL_PATH, map_location=DEVICE)
siamese_model.load_state_dict(checkpoint['model_state_dict'])
siamese_model.eval()
yolo_model = YOLO(YOLO_MODEL_PATH).to(DEVICE)
print("Models loaded successfully.")

# --- ساختارهای داده اصلی (با گالری بهینه‌سازی شده) ---
global_id_counter = 0
# گالری مرکزی: به جای تصاویر، بردار ویژگی (embedding) را برای سرعت بالا ذخیره می‌کند
global_person_gallery = {
    # gid: {"embeddings_gallery": deque, "last_seen_cam": int, ...}
}
# نگاشت TID به GID برای ترک‌های فعال و تایید شده
active_tracks = {i: {} for i in range(len(VIDEO_PATHS))}
# ترک‌های ناشناس: هنوز تصاویر خام را برای ساخت گالری اولیه جمع‌آوری می‌کنند
unidentified_tracks = {i: {} for i in range(len(VIDEO_PATHS))}


# --- توابع کمکی (با توابع بهینه‌سازی شده) ---
def preprocess_image(img):
    transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize(IMG_SIZE), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    return transform(img)

def get_embedding(img, model):
    """یک تصویر خام می‌گیرد و بردار ویژگی آن را برمی‌گرداند."""
    if img.size == 0: return None
    img_tensor = preprocess_image(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        return model.forward_once(img_tensor)

def get_yolo_detections(frame):
    results = yolo_model.predict(frame, conf=CONF_THRESH, classes=[0], verbose=False)
    detections = []
    for result in results:
        boxes, confs = result.boxes.xyxy.cpu().numpy(), result.boxes.conf.cpu().numpy()
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confs.tolist(), CONF_THRESH, IOU_THRESHOLD)
        if len(indices) > 0:
            for i in indices.flatten():
                box = boxes[i]
                if (box[2] - box[0]) >= MIN_BOX_SIZE and (box[3] - box[1]) >= MIN_BOX_SIZE:
                    detections.append(np.concatenate([box, [confs[i]]]))
    return np.array(detections) if detections else np.empty((0, 5))

def compare_embeddings_galleries(embeddings1, embeddings2):
    """دو گالری از بردار ویژگی‌ها را به سرعت مقایسه می‌کند."""
    distances = []
    num_e1 = min(NUM_EMBEDDINGS_COMPARISONS, len(embeddings1))
    num_e2 = min(NUM_EMBEDDINGS_COMPARISONS, len(embeddings2))
    
    # مقایسه متقاطع بین بردارهای ویژگی دو گالری
    for emb1 in list(embeddings1)[-num_e1:]:
        for emb2 in list(embeddings2)[-num_e2:]:
            dist = torch.nn.functional.pairwise_distance(emb1, emb2).item()
            distances.append(dist)
    
    return np.mean(distances) if distances else float('inf')

def find_best_match_for_unidentified(unidentified_track_info, current_cam_id):
    """برای یک فرد ناشناس، بهترین تطابق را در گالری اصلی پیدا می‌کند."""
    # ابتدا گالری تصاویر خام را به گالری بردار ویژگی تبدیل می‌کنیم
    unidentified_embeddings = deque([get_embedding(img, siamese_model) for img in unidentified_track_info["image_gallery"] if img is not None])
    if not unidentified_embeddings: return -1, float('inf')

    min_avg_dist = float('inf')
    best_gid = -1
    
    active_gids_in_current_cam = set(active_tracks[current_cam_id].values())

    for gid, data in global_person_gallery.items():
        if gid in active_gids_in_current_cam:
            continue
        
        dist = compare_embeddings_galleries(unidentified_embeddings, data["embeddings_gallery"])
        if dist < min_avg_dist:
            min_avg_dist = dist
            best_gid = gid
            
    if min_avg_dist < GID_MATCH_THRESHOLD:
        return best_gid, min_avg_dist
    else:
        return -1, min_avg_dist


# --- تابع اصلی پردازش فریم (با منطق نهایی) ---
def process_frame(frame, tracker, cam_id, frame_count):
    global global_id_counter

    detections = get_yolo_detections(frame)
    online_targets = tracker.update(detections, [frame.shape[0], frame.shape[1]], frame.shape[:2])
    
    current_frame_tids = set()
    
    # پردازش ترک‌های آنلاین
    for t in online_targets:
        tid, tlbr = t.track_id, t.tlbr.astype(int)
        current_frame_tids.add(tid)
        person_img = frame[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]]

        # Case 1: ترک از قبل شناسایی شده و فعال است (مسیر بهینه)
        if tid in active_tracks[cam_id]:
            gid = active_tracks[cam_id][tid]
            new_embedding = get_embedding(person_img, siamese_model)
            if new_embedding is not None:
                global_person_gallery[gid]["embeddings_gallery"].append(new_embedding)
            global_person_gallery[gid]["last_seen_cam"] = cam_id
            global_person_gallery[gid]["last_seen_frame"] = frame_count

        # Case 2: ترک در لیست ناشناس‌ها قرار دارد
        elif tid in unidentified_tracks[cam_id]:
            track_info = unidentified_tracks[cam_id][tid]
            track_info["image_gallery"].append(person_img)
            track_info["last_bbox"] = tlbr
            
            # آیا زمان تایید هویت فرا رسیده؟
            if frame_count - track_info["first_seen_frame"] > CONFIRMATION_FRAME_THRESHOLD:
                print(f"Cam {cam_id} | TID {tid}: Confirmation phase started...")
                best_gid, dist = find_best_match_for_unidentified(track_info, cam_id)
                
                # ترفیع به GID موجود
                if best_gid != -1:
                    gid = best_gid
                    active_tracks[cam_id][tid] = gid
                    # استخراج ویژگی‌ها و ادغام در گالری اصلی
                    unidentified_embeddings = [get_embedding(img, siamese_model) for img in track_info["image_gallery"] if img is not None]
                    for emb in unidentified_embeddings: global_person_gallery[gid]["embeddings_gallery"].append(emb)
                    del unidentified_tracks[cam_id][tid]
                    print(f"Cam {cam_id} | TID {tid}: Promoted to GID {gid} (Dist: {dist:.2f})")

                # ترفیع به GID جدید
                else:
                    new_gid = global_id_counter; global_id_counter += 1
                    active_tracks[cam_id][tid] = new_gid
                    global_person_gallery[new_gid] = {
                        "embeddings_gallery": deque([get_embedding(img, siamese_model) for img in track_info["image_gallery"] if img is not None], maxlen=GALLERY_MAX_SIZE),
                        "last_seen_cam": cam_id, "last_seen_frame": frame_count
                    }
                    del unidentified_tracks[cam_id][tid]
                    print(f"Cam {cam_id} | TID {tid}: Promoted to NEW GID {new_gid}")
            
        # Case 3: ترک کاملاً جدید است -> اضافه به لیست ناشناس‌ها برای جمع‌آوری شواهد
        else:
            unidentified_tracks[cam_id][tid] = {
                "image_gallery": deque([person_img], maxlen=GALLERY_MAX_SIZE),
                "first_seen_frame": frame_count, "last_bbox": tlbr
            }

    # --- رسم BBox ها و مدیریت ترک‌های گمشده ---
    # رسم ترک‌های تایید شده
    for tid, gid in active_tracks[cam_id].items():
        # پیدا کردن bbox از آخرین آپدیت tracker
        for t in online_targets:
            if t.track_id == tid:
                tlbr = t.tlbr.astype(int)
                label, color = f"ID:{gid}", (0, 255, 0) # سبز
                cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), color, 2)
                cv2.putText(frame, label, (tlbr[0], tlbr[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                break
    
    # رسم ترک‌های ناشناس
    for tid, info in unidentified_tracks[cam_id].items():
        tlbr = info["last_bbox"]
        label, color = f"ID:?", (255, 165, 0) # نارنجی
        cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), color, 2)
        cv2.putText(frame, label, (tlbr[0], tlbr[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    lost_tids = set(active_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_tids: del active_tracks[cam_id][tid]
        
    lost_unidentified_tids = set(unidentified_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_unidentified_tids: del unidentified_tracks[cam_id][tid]
        
    return frame

# --- حلقه اصلی پردازش ویدیو (بدون تغییر) ---
# --- حلقه اصلی پردازش ویدیو (بدون تغییر) ---
caps = [cv2.VideoCapture(path) for path in VIDEO_PATHS]
# یک رزولوشن استاندارد برای نمایش هر دو ویدیو تعیین می‌کنیم
w, h = 960, 540 
trackers = [BYTETracker(args, frame_rate=30) for _ in range(len(VIDEO_PATHS))]
frame_counts = [0] * len(caps)
out_videos = [cv2.VideoWriter(f"output_cam{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 25, (w, h)) for i in range(len(caps))]

while True:
    frames = []
    rets = []
    # 1. خواندن یک فریم از هر ویدیو
    for cap in caps:
        ret, frame = cap.read()
        rets.append(ret)
        frames.append(frame)

    # اگر یکی از ویدیوها تمام شد، حلقه را متوقف کن
    if not all(rets):
        print("Finished processing one of the videos.")
        break

    processed_frames = []
    # 2. پردازش فریم مربوط به هر دوربین
    for i, frame in enumerate(frames):
        if frame is None:
            continue
        
        frame_counts[i] += 1
        # فراخوانی تابع اصلی پردازشگر
        processed_frame = process_frame(frame, trackers[i], i, frame_counts[i])
        
        # تغییر اندازه فریم پردازش شده به سایز استاندارد
        resized_frame = cv2.resize(processed_frame, (w, h))
        
        # اضافه کردن فریم پردازش شده به لیست
        processed_frames.append(resized_frame)
        
        # ذخیره فریم در فایل ویدیویی خروجی
        out_videos[i].write(resized_frame)

    # 3. ترکیب فریم‌ها و نمایش
    # این شرط برای جلوگیری از خطا در صورت خالی بودن لیست است
    if len(processed_frames) == len(caps):
        # **کلید اصلی**: فریم‌ها را به صورت افقی به هم می‌چسبانیم
        combined_frame = cv2.hconcat(processed_frames)
        
        # نمایش تصویر واحد و ترکیبی
        cv2.imshow("Multi-Camera Live Tracking", combined_frame)
    
    # شرط خروج با زدن کلید 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# آزادسازی منابع
print("Releasing resources...")
for cap in caps:
    cap.release()
for out in out_videos:
    out.release()
cv2.destroyAllWindows()
print("All done.")

Loading models...
Models loaded successfully.
Cam 0 | TID 1: Confirmation phase started...
Cam 0 | TID 1: Promoted to NEW GID 0
Cam 0 | TID 2: Confirmation phase started...
Cam 0 | TID 2: Promoted to NEW GID 1
Cam 1 | TID 3: Confirmation phase started...
Cam 1 | TID 3: Promoted to NEW GID 2
Cam 1 | TID 4: Confirmation phase started...
Cam 1 | TID 4: Promoted to NEW GID 3
Cam 1 | TID 5: Confirmation phase started...
Cam 1 | TID 5: Promoted to GID 1 (Dist: 0.42)
Cam 0 | TID 6: Confirmation phase started...
Cam 0 | TID 6: Promoted to NEW GID 4
Cam 1 | TID 8: Confirmation phase started...
Cam 1 | TID 8: Promoted to GID 0 (Dist: 0.36)
Cam 1 | TID 10: Confirmation phase started...
Cam 1 | TID 10: Promoted to GID 1 (Dist: 0.36)
Cam 0 | TID 9: Confirmation phase started...
Cam 0 | TID 9: Promoted to NEW GID 5
Finished processing one of the videos.
Releasing resources...
All done.


In [2]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from collections import deque
from yolox.tracker.byte_tracker import BYTETracker
from torchvision import transforms, models
from ultralytics import YOLO
from types import SimpleNamespace

# --- Siamese Network Definition (No Change) ---
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet50(weights='IMAGENET1K_V1')
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.embedding = nn.Sequential(nn.Flatten(), nn.Linear(2048, 512), nn.ReLU(), nn.BatchNorm1d(512), nn.Linear(512, 128), nn.Sigmoid())
    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = self.pool(x)
        x = self.embedding(x)
        return x
    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)

# --- CONFIG (No Change) ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIAMESE_MODEL_PATH = r"F:\mtmctwithsiamese\siameselast.pth"
YOLO_MODEL_PATH = "yolov8n.pt"
VIDEO_PATHS = [r"F:\mtmctwithsiamese\22.mp4", r"F:\mtmctwithsiamese\11.mp4"]
IMG_SIZE = (224, 224)
CONF_THRESH = 0.3
MIN_BOX_SIZE = 40
IOU_THRESHOLD = 0.7

# --- Key Hyperparameters (No Change) ---
GALLERY_MAX_SIZE = 50
GID_MATCH_THRESHOLD = 0.48
CONFIRMATION_FRAME_THRESHOLD = 25
NUM_EMBEDDINGS_COMPARISONS = 20

# --- BYTETracker Settings (No Change) ---
args = SimpleNamespace(track_thresh=0.5, track_buffer=90, match_thresh=0.8, min_box_area=MIN_BOX_SIZE*MIN_BOX_SIZE, mot20=False)

# --- Model Loading (No Change) ---
print("Loading models...")
siamese_model = SiameseNetwork().to(DEVICE)
try:
    checkpoint = torch.load(SIAMESE_MODEL_PATH, map_location=DEVICE)
    siamese_model.load_state_dict(checkpoint['model_state_dict'])
except FileNotFoundError:
    print(f"Error: Siamese model not found at {SIAMESE_MODEL_PATH}. Please check the path.")
    exit()
siamese_model.eval()
yolo_model = YOLO(YOLO_MODEL_PATH).to(DEVICE)
print("Models loaded successfully.")

# --- Core Data Structures (No Change) ---
global_id_counter = 0
global_person_gallery = {}
active_tracks = {i: {} for i in range(len(VIDEO_PATHS))}
unidentified_tracks = {i: {} for i in range(len(VIDEO_PATHS))}


# --- Helper Functions (No Change) ---
def preprocess_image(img):
    transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize(IMG_SIZE), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    return transform(img)

def get_embedding(img, model):
    if img.size == 0: return None
    img_tensor = preprocess_image(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        return model.forward_once(img_tensor)

def get_yolo_detections(frame):
    results = yolo_model.predict(frame, conf=CONF_THRESH, classes=[0], verbose=False)
    detections = []
    for result in results:
        boxes, confs = result.boxes.xyxy.cpu().numpy(), result.boxes.conf.cpu().numpy()
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confs.tolist(), CONF_THRESH, IOU_THRESHOLD)
        if len(indices) > 0:
            for i in indices.flatten():
                box = boxes[i]
                if (box[2] - box[0]) >= MIN_BOX_SIZE and (box[3] - box[1]) >= MIN_BOX_SIZE:
                    detections.append(np.concatenate([box, [confs[i]]]))
    return np.array(detections) if detections else np.empty((0, 5))

def compare_embeddings_galleries(embeddings1, embeddings2):
    distances = []
    num_e1 = min(NUM_EMBEDDINGS_COMPARISONS, len(embeddings1))
    num_e2 = min(NUM_EMBEDDINGS_COMPARISONS, len(embeddings2))
    for emb1 in list(embeddings1)[-num_e1:]:
        for emb2 in list(embeddings2)[-num_e2:]:
            dist = torch.nn.functional.pairwise_distance(emb1, emb2).item()
            distances.append(dist)
    return np.mean(distances) if distances else float('inf')

def find_best_match_for_unidentified(unidentified_track_info, current_cam_id):
    unidentified_embeddings = deque([get_embedding(img, siamese_model) for img in unidentified_track_info["image_gallery"] if img is not None])
    if not unidentified_embeddings: return -1, float('inf')
    min_avg_dist = float('inf')
    best_gid = -1
    active_gids_in_current_cam = set(active_tracks[current_cam_id].values())
    for gid, data in global_person_gallery.items():
        if gid in active_gids_in_current_cam:
            continue
        dist = compare_embeddings_galleries(unidentified_embeddings, data["embeddings_gallery"])
        if dist < min_avg_dist:
            min_avg_dist = dist
            best_gid = gid
    if min_avg_dist < GID_MATCH_THRESHOLD:
        return best_gid, min_avg_dist
    else:
        return -1, min_avg_dist


# --- Main Frame Processing Function (MODIFIED) ---
def process_frame(frame, tracker, cam_id, frame_count):
    global global_id_counter

    detections = get_yolo_detections(frame)
    online_targets = tracker.update(detections, [frame.shape[0], frame.shape[1]], frame.shape[:2])
    
    current_frame_tids = set()
    
    # --- Process online targets (Logic unchanged) ---
    for t in online_targets:
        tid, tlbr = t.track_id, t.tlbr.astype(int)
        current_frame_tids.add(tid)
        person_img = frame[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]]

        if tid in active_tracks[cam_id]:
            gid = active_tracks[cam_id][tid]
            new_embedding = get_embedding(person_img, siamese_model)
            if new_embedding is not None:
                global_person_gallery[gid]["embeddings_gallery"].append(new_embedding)
            global_person_gallery[gid]["last_seen_cam"] = cam_id
            global_person_gallery[gid]["last_seen_frame"] = frame_count
        elif tid in unidentified_tracks[cam_id]:
            track_info = unidentified_tracks[cam_id][tid]
            track_info["image_gallery"].append(person_img)
            track_info["last_bbox"] = tlbr
            if frame_count - track_info["first_seen_frame"] > CONFIRMATION_FRAME_THRESHOLD:
                best_gid, dist = find_best_match_for_unidentified(track_info, cam_id)
                if best_gid != -1:
                    gid = best_gid
                    active_tracks[cam_id][tid] = gid
                    unidentified_embeddings = [get_embedding(img, siamese_model) for img in track_info["image_gallery"] if img is not None]
                    for emb in unidentified_embeddings: global_person_gallery[gid]["embeddings_gallery"].append(emb)
                    del unidentified_tracks[cam_id][tid]
                else:
                    new_gid = global_id_counter; global_id_counter += 1
                    active_tracks[cam_id][tid] = new_gid
                    global_person_gallery[new_gid] = {
                        "embeddings_gallery": deque([get_embedding(img, siamese_model) for img in track_info["image_gallery"] if img is not None], maxlen=GALLERY_MAX_SIZE),
                        "last_seen_cam": cam_id, "last_seen_frame": frame_count
                    }
                    del unidentified_tracks[cam_id][tid]
        else:
            unidentified_tracks[cam_id][tid] = {
                "image_gallery": deque([person_img], maxlen=GALLERY_MAX_SIZE),
                "first_seen_frame": frame_count, "last_bbox": tlbr
            }

    # --- *MODIFIED*: Draw BBoxes with ID INSIDE the box ---
    def draw_label(img, bbox, label, color):
        font_scale = 0.8
        font_thickness = 2
        
        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        
        # Position for the background rectangle
        bg_rect_pos = (bbox[0], bbox[1])
        # Position for the text
        text_pos = (bbox[0] + 5, bbox[1] + text_height + 5)
        
        # Draw a filled rectangle as a background for the text
        cv2.rectangle(img, bg_rect_pos, (bbox[0] + text_width + 10, bbox[1] + text_height + baseline + 10), color, cv2.FILLED)
        # Draw the text
        cv2.putText(img, label, text_pos, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

    # Draw confirmed tracks
    for tid, gid in active_tracks[cam_id].items():
        for t in online_targets:
            if t.track_id == tid:
                tlbr = t.tlbr.astype(int)
                label, color = f"ID:{gid}", (0, 180, 0) # Darker Green
                cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), color, 2)
                draw_label(frame, tlbr, label, color)
                break
    
    # Draw unidentified tracks
    for tid, info in unidentified_tracks[cam_id].items():
        tlbr = info["last_bbox"]
        label, color = f"ID:?", (255, 165, 0) # Orange
        cv2.rectangle(frame, (tlbr[0], tlbr[1]), (tlbr[2], tlbr[3]), color, 2)
        draw_label(frame, tlbr, label, color)

    # --- Manage lost tracks (Logic unchanged) ---
    lost_tids = set(active_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_tids: del active_tracks[cam_id][tid]
    lost_unidentified_tids = set(unidentified_tracks[cam_id].keys()) - current_frame_tids
    for tid in lost_unidentified_tids: del unidentified_tracks[cam_id][tid]
        
    return frame

# --- *MODIFIED*: Main video processing loop ---
# --- Handles different video lengths and displays separately with preserved aspect ratio ---
caps = [cv2.VideoCapture(path) for path in VIDEO_PATHS]
trackers = [BYTETracker(args, frame_rate=30) for _ in range(len(VIDEO_PATHS))]
frame_counts = [0] * len(caps)

# Create video writers (optional, for saving output)
output_writers = []
MAX_DISPLAY_WIDTH = 720 # Max width for display windows
for i, cap in enumerate(caps):
    if not cap.isOpened():
        print(f"Error: Could not open video file: {VIDEO_PATHS[i]}")
        continue
    # Calculate output size preserving aspect ratio for saving
    original_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    ratio = MAX_DISPLAY_WIDTH / original_w
    save_w = MAX_DISPLAY_WIDTH
    save_h = int(original_h * ratio)
    output_writers.append(cv2.VideoWriter(f"output_cam_{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 25, (save_w, save_h)))

active_videos = [True] * len(caps)

while any(active_videos):
    # Process one frame from each active video
    for i, cap in enumerate(caps):
        if not active_videos[i]:
            continue

        ret, frame = cap.read()
        
        if ret:
            frame_counts[i] += 1
            processed_frame = process_frame(frame, trackers[i], i, frame_counts[i])
            
            # --- Resize for display, preserving aspect ratio ---
            original_h, original_w = processed_frame.shape[:2]
            ratio = MAX_DISPLAY_WIDTH / original_w
            display_h = int(original_h * ratio)
            display_frame = cv2.resize(processed_frame, (MAX_DISPLAY_WIDTH, display_h))
            
            # Display in its own window
            cv2.imshow(f"Camera {i}", display_frame)
            
            # Save the resized frame to output file
            if i < len(output_writers):
                output_writers[i].write(display_frame)

        else:
            # This video has ended
            active_videos[i] = False
            print(f"Video for Camera {i} has finished.")

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Resource Cleanup ---
print("Releasing resources...")
for cap in caps:
    cap.release()
for out in output_writers:
    out.release()
cv2.destroyAllWindows()
print("All done.")


Loading models...
Models loaded successfully.
Video for Camera 1 has finished.
Video for Camera 0 has finished.
Releasing resources...
All done.
